# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-07 13:30:13] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37971, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=162359794, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fals

[2025-04-07 13:30:46 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-07 13:30:46 TP0] Init torch distributed begin.
[2025-04-07 13:30:46 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-07 13:30:46 TP0] Load weight begin. avail mem=78.81 GB


[2025-04-07 13:30:47 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-07 13:30:55 TP0] Using model weights format ['*.safetensors']


[2025-04-07 13:30:57 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.15it/s]

[2025-04-07 13:30:58 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=60.75 GB, mem usage=18.06 GB.
[2025-04-07 13:30:58 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-04-07 13:30:58 TP0] Memory pool end. avail mem=59.90 GB


[2025-04-07 13:31:00 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-07 13:31:01] INFO:     Started server process [197419]
[2025-04-07 13:31:01] INFO:     Waiting for application startup.
[2025-04-07 13:31:01] INFO:     Application startup complete.
[2025-04-07 13:31:01] INFO:     Uvicorn running on http://0.0.0.0:37971 (Press CTRL+C to quit)


[2025-04-07 13:31:01] INFO:     127.0.0.1:53750 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-07 13:31:02] INFO:     127.0.0.1:53754 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-07 13:31:02 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-04-07 13:31:07 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-04-07 13:31:07] INFO:     127.0.0.1:53762 - "POST /generate HTTP/1.1" 200 OK
[2025-04-07 13:31:07] The server is fired up and ready to roll!


[2025-04-07 13:31:08 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 6.55, #queue-req: 0, 


[2025-04-07 13:31:08 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 189.39, #queue-req: 0, 
[2025-04-07 13:31:08] INFO:     127.0.0.1:53764 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-04-07 13:31:08] INFO:     127.0.0.1:53770 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-04-07 13:31:08] INFO:     127.0.0.1:53774 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-04-07 13:31:08 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-07 13:31:09] INFO:     127.0.0.1:53776 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-04-07 13:31:09] INFO:     127.0.0.1:53786 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-04-07 13:31:09] INFO:     127.0.0.1:53790 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-04-07 13:31:09 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-04-07 13:31:09] Start update_weights. Load format=auto
[2025-04-07 13:31:09 TP0] Update engine weights online from disk begin. avail mem=59.23 GB


[2025-04-07 13:31:10 TP0] Using model weights format ['*.safetensors']


[2025-04-07 13:31:11 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]

[2025-04-07 13:31:12 TP0] Update weights end.
[2025-04-07 13:31:12 TP0] Cache flushed successfully!
[2025-04-07 13:31:12] INFO:     127.0.0.1:53802 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-04-07 13:31:12] Start update_weights. Load format=auto
[2025-04-07 13:31:12 TP0] Update engine weights online from disk begin. avail mem=59.23 GB


[2025-04-07 13:31:13 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-04-07 13:31:13] INFO:     127.0.0.1:42298 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-07 13:31:34] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=30688, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=223242069, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fa

[2025-04-07 13:31:40] Downcasting torch.float32 to torch.float16.


[2025-04-07 13:31:57 TP0] Downcasting torch.float32 to torch.float16.


[2025-04-07 13:31:58 TP0] Overlap scheduler is disabled for embedding models.


[2025-04-07 13:31:58 TP0] Downcasting torch.float32 to torch.float16.
[2025-04-07 13:31:58 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-07 13:31:58 TP0] Init torch distributed begin.
[2025-04-07 13:31:58 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-07 13:31:58 TP0] Load weight begin. avail mem=78.81 GB


...

Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.43s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:11<00:00,  1.58s/it]

[2025-04-07 13:32:13 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=64.38 GB, mem usage=14.43 GB.
[2025-04-07 13:32:13 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-04-07 13:32:13 TP0] Memory pool end. avail mem=63.00 GB


[2025-04-07 13:32:13 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-07 13:32:14] INFO:     Started server process [199263]
[2025-04-07 13:32:14] INFO:     Waiting for application startup.
[2025-04-07 13:32:14] INFO:     Application startup complete.
[2025-04-07 13:32:14] INFO:     Uvicorn running on http://0.0.0.0:30688 (Press CTRL+C to quit)
[2025-04-07 13:32:14] INFO:     127.0.0.1:33470 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-07 13:32:15] INFO:     127.0.0.1:33478 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-07 13:32:15 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-07 13:32:16] INFO:     127.0.0.1:33482 - "POST /encode HTTP/1.1" 200 OK
[2025-04-07 13:32:16] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-04-07 13:32:19 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-07 13:32:19] INFO:     127.0.0.1:33486 - "POST /encode HTTP/1.1" 200 OK


In [12]:
terminate_process(embedding_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-07 13:32:40] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=33975, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=488690503, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=

[2025-04-07 13:33:05 TP0] Overlap scheduler is disabled for embedding models.


[2025-04-07 13:33:06 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-07 13:33:06 TP0] Init torch distributed begin.
[2025-04-07 13:33:06 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-07 13:33:06 TP0] Load weight begin. avail mem=78.81 GB


[2025-04-07 13:33:07 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-07 13:33:09 TP0] Using model weights format ['*.safetensors']


...

[2025-04-07 13:33:14 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-07 13:33:15] INFO:     Started server process [200395]
[2025-04-07 13:33:15] INFO:     Waiting for application startup.
[2025-04-07 13:33:15] INFO:     Application startup complete.
[2025-04-07 13:33:15] INFO:     Uvicorn running on http://0.0.0.0:33975 (Press CTRL+C to quit)


[2025-04-07 13:33:15] INFO:     127.0.0.1:32828 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-07 13:33:16] INFO:     127.0.0.1:32844 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-07 13:33:16 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-07 13:33:17] INFO:     127.0.0.1:32846 - "POST /encode HTTP/1.1" 200 OK
[2025-04-07 13:33:17] The server is fired up and ready to roll!


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-04-07 13:33:29 TP0] Prefill batch. #new-seq: 1, #new-token: 68, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-07 13:33:29 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 62, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-07 13:33:29] INFO:     127.0.0.1:41776 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)

## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

In [16]:
expert_record_server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-07 13:33:50] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=35275, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=221393702, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_

[2025-04-07 13:34:13 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-07 13:34:13 TP0] Init torch distributed begin.
[2025-04-07 13:34:13 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-07 13:34:13 TP0] Load weight begin. avail mem=78.81 GB


[2025-04-07 13:34:14 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-07 13:34:16 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.33it/s]

[2025-04-07 13:34:31 TP0] Load weight end. type=Qwen2MoeForCausalLM, dtype=torch.bfloat16, avail mem=51.99 GB, mem usage=26.82 GB.
[2025-04-07 13:34:31 TP0] KV Cache is allocated. #tokens: 20480, K size: 1.88 GB, V size: 1.88 GB
[2025-04-07 13:34:31 TP0] Memory pool end. avail mem=48.08 GB


[2025-04-07 13:34:32 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192


[2025-04-07 13:34:33] INFO:     Started server process [201460]
[2025-04-07 13:34:33] INFO:     Waiting for application startup.
[2025-04-07 13:34:33] INFO:     Application startup complete.
[2025-04-07 13:34:33] INFO:     Uvicorn running on http://0.0.0.0:35275 (Press CTRL+C to quit)


[2025-04-07 13:34:33] INFO:     127.0.0.1:50946 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-07 13:34:34] INFO:     127.0.0.1:50956 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-07 13:34:34 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [17]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("Content of dumped record:")
    for line in f:
        print_highlight(line.strip())

[2025-04-07 13:34:38 TP0] Using default MoE config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-mc-gpu-0/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json


[2025-04-07 13:34:39 TP0] Resetting expert distribution record...
[2025-04-07 13:34:39] INFO:     127.0.0.1:50970 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-07 13:34:39 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-07 13:34:40] INFO:     127.0.0.1:50960 - "POST /generate HTTP/1.1" 200 OK
[2025-04-07 13:34:40] The server is fired up and ready to roll!
[2025-04-07 13:34:40] INFO:     127.0.0.1:50978 - "POST /generate HTTP/1.1" 200 OK


[2025-04-07 13:34:40] INFO:     127.0.0.1:59350 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-07 13:34:40 TP0] Resetting expert distribution record...
[2025-04-07 13:34:40] INFO:     127.0.0.1:59362 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [18]:
terminate_process(expert_record_server_process)

## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [19]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-07 13:35:02] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=True, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=36435, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=884841171, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fal

[2025-04-07 13:35:34 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-07 13:35:34 TP0] Init torch distributed begin.
[2025-04-07 13:35:34 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-07 13:35:34 TP0] Load weight begin. avail mem=78.81 GB


[2025-04-07 13:35:35 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-07 13:35:42 TP0] Using model weights format ['*.safetensors']


[2025-04-07 13:35:49 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.19it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.18it/s]

[2025-04-07 13:35:50 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.37 GB, mem usage=2.44 GB.
[2025-04-07 13:35:50 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-04-07 13:35:50 TP0] Memory pool end. avail mem=75.51 GB


[2025-04-07 13:35:50 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-04-07 13:35:50] INFO:     Started server process [202545]
[2025-04-07 13:35:50] INFO:     Waiting for application startup.
[2025-04-07 13:35:50] INFO:     Application startup complete.
[2025-04-07 13:35:50] INFO:     Uvicorn running on http://127.0.0.1:36435 (Press CTRL+C to quit)


[2025-04-07 13:35:50] INFO:     127.0.0.1:58864 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-07 13:35:51] INFO:     127.0.0.1:58870 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-07 13:35:51 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["output_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-04-07 13:35:56 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-04-07 13:35:57] INFO:     127.0.0.1:58878 - "POST /generate HTTP/1.1" 200 OK
[2025-04-07 13:35:57] The server is fired up and ready to roll!


[2025-04-07 13:35:57 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 7.05, #queue-req: 0, 


[2025-04-07 13:35:57 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 173.70, #queue-req: 0, 
[2025-04-07 13:35:57] INFO:     127.0.0.1:58880 - "POST /generate HTTP/1.1" 200 OK


In [21]:
terminate_process(tokenizer_free_server_process)